#  Подготовка

In [1]:
# Рисовать графики сразу же
%matplotlib inline

import re # Регулярные выражения
from sklearn.neural_network import MLPRegressor
import pandas as pd # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import datetime # Импортируем модуль datetime для обработки данных в формете даты
import numpy as np # Импортируем библиотеку numpy
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import catboost as cb

In [2]:
data = pd.read_csv('Данные объявлений об эскорт-услугах в Москве(214k).csv', delimiter=',') # Импортируем базу данных

In [3]:
data = data.drop(data.columns[0],axis = 1) # Удаляем первый столбец с продублированными индексами

# Парсинг данных

In [4]:
data = data.drop('DateUpdate', axis=1)
data = data.astype({"Age": np.uint8})
data = data.astype({"Boobs": np.uint8})
data = data.astype({"Height": np.uint8})
data = data.astype({"Size": np.uint8})
data = data.astype({"Weight": np.uint8})
data = data.astype({"Weight": np.uint8})
data = data.astype({"Price_USD": np.uint8})
data

,Age,Boobs,Height,Size,Metro,Weight,Price_USD
0,27,3,173,46,"Алтуфьево, Медведково, Отрадное",57,22
1,27,3,168,46,Кунцевская,65,30
2,25,4,165,52,"Кутузовская, Студенческая, Киевская",52,223
3,21,2,170,42,Тимирязевская,60,22
4,35,4,180,46,Проспект Мира,70,52
...,...,...,...,...,...,...,...
214120,22,5,172,44,"Бульвар Дмитрия Донского, Южная, Ясенево",52,74
214121,20,3,180,38,"Академическая, Ленинский проспект, Университет",60,148
214122,22,2,171,44,NaN,64,22
214123,26,3,170,44,"Академическая, Калужская, Ленинский проспект",54,134


In [5]:
#Преобразование стобца "Метро" в строковый тип

data.dropna(subset=["Metro"], inplace=True)
data["Metro"] = data["Metro"].astype("category")
data["Metro"].dtype

CategoricalDtype(categories=['Авиамоторная', 'Авиамоторная, Автозаводская',
                  'Авиамоторная, Автозаводская, Академическая',
                  'Авиамоторная, Автозаводская, Александровский сад',
                  'Авиамоторная, Автозаводская, Алексеевская',
                  'Авиамоторная, Автозаводская, Арбатская',
                  'Авиамоторная, Автозаводская, Багратионовская',
                  'Авиамоторная, Автозаводская, Ленинский проспект',
                  'Авиамоторная, Автозаводская, Новогиреево',
                  'Авиамоторная, Автозаводская, Новокосино',
                  ...
                  'Ясенево, Чертановская, Академическая',
                  'Ясенево, Шаболовская, Теплый стан',
                  'Ясенево, Шипиловская, Электрозаводская',
                  'Ясенево, Щукинская, Чкаловская', 'Ясенево, Юго-Западная',
                  'Ясенево, Юго-Западная, Теплый стан',
                  'Ясенево, Юго-Западная, Улица Ак. Янгеля',
                  'Я

In [6]:
# Поиск строк с несколькими станциями
data["Metro_list"] = data["Metro"].str.split(",")
multiple_stations = data[data["Metro_list"].str.len() > 1]

# Удаление строк
data = data.drop(multiple_stations.index)


data

,Age,Boobs,Height,Size,Metro,Weight,Price_USD,Metro_list
1,27,3,168,46,Кунцевская,65,30,[Кунцевская]
3,21,2,170,42,Тимирязевская,60,22,[Тимирязевская]
4,35,4,180,46,Проспект Мира,70,52,[Проспект Мира]
12,31,3,180,44,Сухаревская,56,59,[Сухаревская]
13,25,5,170,46,Алексеевская,65,45,[Алексеевская]
...,...,...,...,...,...,...,...,...
214112,23,2,175,44,Ленинский проспект,58,36,[Ленинский проспект]
214114,23,3,168,44,Перово,54,37,[Перово]
214116,21,2,165,42,Марьино,50,52,[Марьино]
214117,27,2,170,44,Профсоюзная,56,38,[Профсоюзная]


In [7]:
data = data.drop('Metro_list', axis=1)
data

,Age,Boobs,Height,Size,Metro,Weight,Price_USD
1,27,3,168,46,Кунцевская,65,30
3,21,2,170,42,Тимирязевская,60,22
4,35,4,180,46,Проспект Мира,70,52
12,31,3,180,44,Сухаревская,56,59
13,25,5,170,46,Алексеевская,65,45
...,...,...,...,...,...,...,...
214112,23,2,175,44,Ленинский проспект,58,36
214114,23,3,168,44,Перово,54,37
214116,21,2,165,42,Марьино,50,52
214117,27,2,170,44,Профсоюзная,56,38


# Создание модели и обучение

In [8]:
# Преобразование типов данных
data["Age"] = data["Age"].astype("float")
data["Boobs"] = data["Boobs"].astype("float")
data["Height"] = data["Height"].astype("float")
data["Size"] = data["Size"].astype("float")
data["Weight"] = data["Weight"].astype("float")

# Преобразование категориального столбца
data["Metro"] = data["Metro"].astype("category")

In [9]:
from sklearn.preprocessing import LabelEncoder

# Создаем LabelEncoder
le = LabelEncoder()

# Кодируем категориальную переменную
data["Metro"] = le.fit_transform(data["Metro"])

data

,Age,Boobs,Height,Size,Metro,Weight,Price_USD
1,27.0,3.0,168.0,46.0,85,65.0,30
3,21.0,2.0,170.0,42.0,192,60.0,22
4,35.0,4.0,180.0,46.0,157,70.0,52
12,31.0,3.0,180.0,44.0,185,56.0,59
13,25.0,5.0,170.0,46.0,6,65.0,45
...,...,...,...,...,...,...,...
214112,23.0,2.0,175.0,44.0,90,58.0,36
214114,23.0,3.0,168.0,44.0,136,54.0,37
214116,21.0,2.0,165.0,42.0,101,50.0,52
214117,27.0,2.0,170.0,44.0,158,56.0,38


In [10]:
from sklearn.model_selection import train_test_split
# Разделение данных на 70% обучающих и 30% тестовых
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data, data["Price_USD"], test_size=0.25)

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Создание объекта GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=0)

# Обучение модели
model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=0)

In [13]:
# Прогнозы
y_pred = model.predict(X_test)

In [14]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


# Оценка качества
print("R^2:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))

R^2: 0.9999963781920664
RMSE: 0.05664889141598421
MAE: 0.008731155122576386


In [15]:
modelcb = cb.CatBoostRegressor(task_type='GPU',
                              iterations=100,
                              learning_rate=0.1,
                              depth=10,
                              loss_function='RMSE')

loss_function='RMSE'

In [17]:
modelcb.fit(X_train, y_train)

0:	learn: 27.0008351	total: 29.8ms	remaining: 2.95s
1:	learn: 24.3910192	total: 35.4ms	remaining: 1.74s
2:	learn: 22.0336185	total: 40.9ms	remaining: 1.32s
3:	learn: 19.9052911	total: 45.9ms	remaining: 1.1s
4:	learn: 17.9779886	total: 51ms	remaining: 969ms
5:	learn: 16.2373778	total: 55.8ms	remaining: 873ms
6:	learn: 14.6712236	total: 61.2ms	remaining: 814ms
7:	learn: 13.2548300	total: 66.4ms	remaining: 764ms
8:	learn: 11.9775057	total: 71.7ms	remaining: 725ms
9:	learn: 10.8107300	total: 76.9ms	remaining: 692ms
10:	learn: 9.7819410	total: 82.7ms	remaining: 669ms
11:	learn: 8.8423418	total: 88ms	remaining: 645ms
12:	learn: 7.9932160	total: 93.1ms	remaining: 623ms
13:	learn: 7.2212270	total: 98.2ms	remaining: 603ms
14:	learn: 6.5302990	total: 103ms	remaining: 585ms
15:	learn: 5.9053709	total: 108ms	remaining: 568ms
16:	learn: 5.3447097	total: 113ms	remaining: 554ms
17:	learn: 4.8224008	total: 119ms	remaining: 544ms
18:	learn: 4.3525643	total: 127ms	remaining: 543ms
19:	learn: 3.9386644	t

In [18]:
modelcb.save_model('my_model.cbm')

In [19]:
modelcb.load_model('my_model.cbm')

In [20]:
y_predcb = modelcb.predict(X_test)

In [23]:
# Оценка качества
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error, r2_score

print("MSE:", mean_squared_error(y_test, y_predcb))
print("R^2:", r2_score(y_test, y_predcb))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_predcb)))
print("MAE:", mean_absolute_error(y_test, y_predcb))
print("MAPE:", mean_absolute_percentage_error(y_test, y_predcb))

MSE: 0.012328308095267521
R^2: 0.9999860861901411
RMSE: 0.11103291446804196
MAE: 0.04603794013087972
MAPE: 0.0010697193623519054
